In [1]:
# %%
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle
# %%

procdir = "C:/Users/jmc753/Work/occ-coseismic/data/sz_solutions/"
branch = "NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTA3NzE0"
rate_file = os.path.join(procdir, branch, "solution", "rates.csv")
indicies_file = os.path.join(procdir, branch, "ruptures", "indices.csv")
slip_file = os.path.join(procdir, branch, "ruptures", "average_slips.csv")
geometry_file = "C:/Users/jmc753/Work/occ-coseismic/subduction/discretised_hikkerk/sz_all_rectangle_outlines.geojson"
event_ids = [1733]




In [2]:
for event_id in event_ids:
    patch_gdf = gpd.read_file(geometry_file)
    patch_gdf['slip'] = 0.

    av_slip_df = pd.read_csv(slip_file, index_col='Rupture Index')
    av_slip = av_slip_df.loc[event_id, 'Average Slip (m)']

    indices_df = pd.read_csv(indicies_file, index_col='Rupture Index')
    indicies = np.array(indices_df.loc[event_id][1:].dropna(), dtype=int)
    patch_gdf = patch_gdf.loc[indicies]
    patch_gdf['slip'] = np.log10(av_slip)

    rupture_dir = 'Z:\\McGrath\\HikurangiFakeQuakes\\hikkerk3D_hires\\output\\ruptures\\'
    rupture_png_dir = os.path.abspath(os.path.dirname(rupture_dir) + '/..\\rupture_pngs\\')

    fig, ax = pickle.load(open(f"{rupture_png_dir}/temp.pkl", "rb"))
    bounds = (1500000, 5250000, 3000000, 7000000)
    min_slip, max_slip = 0.1, 50
    ticks = np.arange(np.log10(min_slip), np.log10(max_slip), 1)
    ticks = list(10**ticks) + [max_slip]
    patch_gdf.plot(column=patch_gdf['slip'], ax=ax['main_figure'], cmap='magma', vmin=np.log10(min_slip), 
                   vmax=np.log10(max_slip), legend=True, 
                   legend_kwds={'label': "Slip (m)", 'orientation': 'vertical', 'ticks': [np.log10(tick) for tick in ticks]})
    cbar = ax['main_figure'].get_figure().get_axes()[-1]
    cbar.set_yticklabels([f'{tick:.2f}' for tick in ticks])

    coastfile = "C:/Users/jmc753/Work/occ-coseismic/data/coastline/nz_coastline.geojson"
    coastline = gpd.read_file(coastfile)
    coastline.plot(ax=ax["main_figure"], color="k", linewidth=0.5)
    #plt.title(f"ev{event_id}")
    fig.savefig(f"{rupture_png_dir}/NSHM_ev{event_id}.png", dpi=300)
    fig.savefig(f"{rupture_png_dir}/NSHM_ev{event_id}.pdf", dpi=300, format='pdf')
    plt.close()